## Download & install packages and models

In [1]:
import sys
sys.path.append('..')

In [2]:
from typing import List
import sentence_embedding_evaluation_german as seeg
import torch
import numpy as np

/Users/uh/projects/paper-293/sentence-embedding-evaluation-german/.venv/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
%%capture
!pip install fasttext
# https://www.deepset.ai/german-word-embeddings
!wget -nc "https://s3.eu-central-1.amazonaws.com/int-emb-fasttext-de-wiki/20180917/model.bin"

In [4]:
import fasttext.util

## Specify & run sentence embedding evaluation

In [5]:
%%time
# (1) Instantiate your Embedding model
ft = fasttext.load_model('model.bin')

# (2) Specify the preprocessing
def preprocesser(batch: List[str], params: dict=None) -> List[List[float]]:
    """ Specify your embedding or pretrained encoder here
    Paramters:
    ----------
    params : dict
        The params dictionary
    batch : List[str]
        A list of sentence as string
    Returns:
    --------
    List[List[float]]
        A list of embedding vectors
    """
    features = np.vstack([
        ft.get_sentence_vector(str(sent).replace("\n", " ")) for sent in batch])
    return features

# (3) Training settings
params = {
    'datafolder': '../datasets',
    'bias': True,
    'balanced': True,
    'batch_size': 128, 
    'num_epochs': 500,
    # 'early_stopping': True,
    # 'split_ratio': 0.1,  # if early_stopping=True
    # 'patience': 10,  # if early_stopping=True
}

# (4) Specify downstream tasks
downstream_tasks = ['FCLAIM', 'VMWE', 'OL19-C', 'ABSD-2', 'MIO-P', 'ARCHI', 'LSDC']

# (5) Run experiments
results = seeg.evaluate(downstream_tasks, preprocesser, **params)

CPU times: user 17min 39s, sys: 1min 2s, total: 18min 41s
Wall time: 19min 4s


In [7]:
# save results
import json
with open("seeg-results-deepset.json", 'w') as fp:
    json.dump(results, fp, indent=2)

## Display the results

In [8]:
print("Task | Epochs | N train | N test")
for res in results:
    print(f"{res['task']:>7s}: {res['epochs']:5d} {res['train']['num']:6d} {res['test']['num']:6d}")

Task | Epochs | N train | N test
 FCLAIM:   500   3244    944
   VMWE:   500   6652   1447
 OL19-C:   500   1921    930
 ABSD-2:   500  19432   2555
  MIO-P:   500   4668   4668
  ARCHI:   500  18809   4743
   LSDC:   500  74140   8602


In [9]:
metric = 'acc'  # 'acc', 'acc-balanced'
print('  Task | train | test')
for res in results:
    print(f"{res['task']:>7s}: {res['train'][metric]:6.3f} {res['test'][metric]:6.3f}")

  Task | train | test
 FCLAIM:  0.616  0.597
   VMWE:  0.837  0.838
 OL19-C:  0.778  0.752
 ABSD-2:  0.546  0.533
  MIO-P:  0.688  0.694
  ARCHI:  0.521  0.416
   LSDC:  0.599  0.458


In [10]:
metric = 'acc-balanced'  # 'acc', 'acc-balanced'
print('  Task | train | test')
for res in results:
    print(f"{res['task']:>7s}: {res['train'][metric]:6.3f} {res['test'][metric]:6.3f}")

  Task | train | test
 FCLAIM:  0.630  0.607
   VMWE:  0.678  0.688
 OL19-C:  0.600  0.551
 ABSD-2:  0.534  0.471
  MIO-P:  0.741  0.743
  ARCHI:  0.521  0.416
   LSDC:  0.295  0.303


In [11]:
metric = 'f1'  # 'f1', 'f1-balanced'
print('  Task | train | test')
for res in results:
    print(f"{res['task']:>7s}: {res['train'][metric]:6.3f} {res['test'][metric]:6.3f}")

  Task | train | test
 FCLAIM:  0.616  0.597
   VMWE:  0.837  0.838
 OL19-C:  0.778  0.752
 ABSD-2:  0.546  0.533
  MIO-P:  0.688  0.694
  ARCHI:  0.521  0.416
   LSDC:  0.599  0.458


In [12]:
metric = 'f1-balanced'  # 'f1', 'f1-balanced'
print('  Task | train | test')
for res in results:
    print(f"{res['task']:>7s}: {res['train'][metric]:6.3f} {res['test'][metric]:6.3f}")

  Task | train | test
 FCLAIM:  0.606  0.585
   VMWE:  0.691  0.701
 OL19-C:  0.584  0.545
 ABSD-2:  0.458  0.418
  MIO-P:  0.628  0.632
  ARCHI:  0.520  0.408
   LSDC:  0.274  0.239


In [13]:
# class label distributions (inference)
for res in results:
    print(res['task'], res['test']['num'])
    print(res['test']['distr-test'])
    print(res['test']['distr-pred'])

FCLAIM 944
{'0': '630', '1': '314'}
{'1': '464', '0': '480'}
VMWE 1447
{'1': '256', '0': '1191'}
{'1': '213', '0': '1234'}
OL19-C 930
{'0': '796', '1': '134'}
{'0': '761', '1': '169'}
ABSD-2 2555
{'1': '1670', '0': '780', '2': '105'}
{'0': '1113', '1': '1056', '2': '386'}
MIO-P 4668
{'1': '812', '0': '3856'}
{'1': '1941', '0': '2727'}
ARCHI 4743
{'2': '1177', '3': '1176', '1': '1199', '0': '1191'}
{'2': '1555', '3': '726', '0': '1129', '1': '1333'}
LSDC 8602
{'13': '280', '6': '346', '12': '940', '3': '925', '2': '944', '1': '934', '11': '931', '0': '453', '10': '511', '5': '924', '4': '65', '8': '923', '9': '83', '7': '343'}
{'1': '2026', '12': '1381', '3': '1506', '11': '1198', '5': '1392', '8': '1097', '7': '2'}
